**<center><font size="25"> 2020 Elections - House of Representatives </font></center>**

**<center> By: Michael Dunphy </center>**
**<center> August 2020 </center>**

<figure>
<img src="https://pbs.twimg.com/media/Eg4GITHWkAA0B4h?format=jpg&name=large" width="800" style ="float:center;">
</figure>

## Introduction

A dashboard of forecasts of the House of Representatives' Elections for 2020.

Visual can be found here: https://public.tableau.com/profile/michael.dunphy8764#!/vizhome/2020Elections-USHouseofRepresentatives/MainDashboard?publish=yes

Data Source:  https://en.wikipedia.org/wiki/2020_United_States_House_of_Representatives_election_ratings#cite_note-CPRratings1-3

Visual also makes use of district data from here: https://cookpolitical.com/pvi-map-and-district-list 

Below is the code used to collect and modify the data to produce the Tableau visual.

## Code

The program makes use of several libraries including:
* [Pandas](https://pandas.pydata.org/docs/)
* [Requests](https://requests.readthedocs.io/en/master/)
* [BeatifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [1]:
!pip install lxml
!pip install html5lib
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# Accesses wiki page with forcast data from Cook, IE, Sabato, Politico, Daily Kos, RCP, and Niskanen
r = requests.get('https://en.wikipedia.org/wiki/2020_United_States_House_of_Representatives_election_ratings#cite_note-CPRratings1-3')
root = BeautifulSoup(r.content)
a = root.prettify()

li = pd.read_html(a)
ratings = pd.DataFrame(li[0])

# Creates data frame of dates column is updated on Wiki
updated = pd.DataFrame(ratings.columns.values[4:])
updated.to_csv("updated.csv", index= False)

# Ratings will be the main table cleaned and used in this visual
ratings.columns = ["District", "2017 CPVI", "Incumbent", "Last result", "Cook", "IE", "Sabato", 
                      "Politico", "Daily Kos", "RCP", "Niskanen"]

overall = ratings[-2:]

ratings.drop(ratings.index[-2:], inplace= True) 

ratings.head()

,District,2017 CPVI,Incumbent,Last result,Cook,IE,Sabato,Politico,Daily Kos,RCP,Niskanen
0,Alaska at-large,R+9,Don Young (R),53.1% R,Lean R,Likely R,Likely R,Likely R,Likely R,Likely R,Lean R
1,Arizona 1,R+2,Tom O'Halleran (D),53.8% D,Lean D,Likely D,Likely D,Lean D,Lean D,Lean D,Safe D
2,Arizona 2,R+1,Ann Kirkpatrick (D),54.7% D,Safe D,Safe D,Likely D,Likely D,Safe D,Lean D,Safe D
3,Arizona 6,R+9,David Schweikert (R),55.2% R,Tossup,Lean R,Lean R,Lean R,Lean R,Lean R,Tossup
4,Arizona 8,R+13,Debbie Lesko (R),55.5% R,Safe R,Safe R,Safe R,Likely R,Safe R,Safe R,Safe R


In [3]:
# Used to count the number of groupings for each available district
def get_count(row, prediction):
    count = 0
    column_names = ratings.columns.values[4:11]
    
    for col in column_names:     
        if prediction in row[col]:
            count += 1
    return count

In [4]:
# Used to get consensus ruling for the state of the race for that district.
def get_consensus(row):
    m = 0
    column_names = ratings.columns.values[11:20]
    
    for col in column_names:
        
        if row[col] >= m:
            m = ratings.iloc[index][col]
            prediction = col
            
    return prediction, m

In [5]:
# Adds numbers of groups to each district
safe_r = []
likely_r = []
lean_r = []
tilt_r = []
toss = []
tilt_d = []
lean_d = []
likely_d = []
safe_d = []

for index, row in ratings.iterrows():
    safe_r.append(get_count(row, "Safe R"))
                  
    likely_r.append(get_count(row, "Likely R"))
                    
    lean_r_c = 0 
    lean_r_c += (get_count(row, "Leans R"))
    lean_r_c += (get_count(row, "Lean R"))
    lean_r.append(lean_r_c)
                 
    tilt_r.append(get_count(row, "Tilt R"))
                  
    toss.append(get_count(row, "Tossup"))
                
    safe_d.append(get_count(row, "Safe D"))
                  
    likely_d.append(get_count(row, "Likely D"))
                    
    lean_d.append(get_count(row, "Lean D"))
                  
    tilt_d.append(get_count(row, "Tilt D"))

ratings["Safe R"] = safe_r
ratings["Likely R"] = likely_r
ratings["Lean R"] = lean_r
ratings["Tilt R"] = tilt_r
ratings["Tossup"] = toss
ratings["Safe D"] = safe_d
ratings["Likely D"] = likely_d
ratings["Lean D"] = lean_d
ratings["Tilt D"] = tilt_d

ratings.head()

,District,2017 CPVI,Incumbent,Last result,Cook,IE,Sabato,Politico,Daily Kos,RCP,Niskanen,Safe R,Likely R,Lean R,Tilt R,Tossup,Safe D,Likely D,Lean D,Tilt D
0,Alaska at-large,R+9,Don Young (R),53.1% R,Lean R,Likely R,Likely R,Likely R,Likely R,Likely R,Lean R,0,5,2,0,0,0,0,0,0
1,Arizona 1,R+2,Tom O'Halleran (D),53.8% D,Lean D,Likely D,Likely D,Lean D,Lean D,Lean D,Safe D,0,0,0,0,0,1,2,4,0
2,Arizona 2,R+1,Ann Kirkpatrick (D),54.7% D,Safe D,Safe D,Likely D,Likely D,Safe D,Lean D,Safe D,0,0,0,0,0,4,2,1,0
3,Arizona 6,R+9,David Schweikert (R),55.2% R,Tossup,Lean R,Lean R,Lean R,Lean R,Lean R,Tossup,0,0,5,0,2,0,0,0,0
4,Arizona 8,R+13,Debbie Lesko (R),55.5% R,Safe R,Safe R,Safe R,Likely R,Safe R,Safe R,Safe R,6,1,0,0,0,0,0,0,0


In [6]:
# Adds consensus ruling, max number of same groupings, and confidence of consensus ruling
prediction = []
m = []

for index, row in ratings.iterrows():
    p, em = get_consensus(row)
    prediction.append(p)
    m.append(em)

ratings["Consensus"] = prediction
ratings["Max"] = m
ratings["Confidence"] = ratings["Max"] / 7
ratings.head()

,District,2017 CPVI,Incumbent,Last result,Cook,IE,Sabato,Politico,Daily Kos,RCP,...,Lean R,Tilt R,Tossup,Safe D,Likely D,Lean D,Tilt D,Consensus,Max,Confidence
0,Alaska at-large,R+9,Don Young (R),53.1% R,Lean R,Likely R,Likely R,Likely R,Likely R,Likely R,...,2,0,0,0,0,0,0,Likely R,5,0.714286
1,Arizona 1,R+2,Tom O'Halleran (D),53.8% D,Lean D,Likely D,Likely D,Lean D,Lean D,Lean D,...,0,0,0,1,2,4,0,Lean D,4,0.571429
2,Arizona 2,R+1,Ann Kirkpatrick (D),54.7% D,Safe D,Safe D,Likely D,Likely D,Safe D,Lean D,...,0,0,0,4,2,1,0,Safe D,4,0.571429
3,Arizona 6,R+9,David Schweikert (R),55.2% R,Tossup,Lean R,Lean R,Lean R,Lean R,Lean R,...,5,0,2,0,0,0,0,Lean R,5,0.714286
4,Arizona 8,R+13,Debbie Lesko (R),55.5% R,Safe R,Safe R,Safe R,Likely R,Safe R,Safe R,...,0,0,0,0,0,0,0,Safe R,6,0.857143


In [7]:
# Access PVI data for missing districts not included in wiki page (districts that all forcasters beleive to 
# be either safe D/R)
pvi = pd.read_csv('cpr-pvi.txt')

# Clean up two cases in which the district has a vacant incumbent
pvi["Incumbent"] = pvi["Incumbent"].str.replace("VACANT ", "")
pvi["Incumbent"] = pvi["Incumbent"].str.replace("\(Marino\)", "Marino")
pvi["Incumbent"] = pvi["Incumbent"].str.replace("\(Jones\)", "Jones")

pvi.head()

,Dist,Incumbent,PVI,Clinton %,Trump %,Obama %,Romney %
0,AL-01,Bradley Byrne (R),R+15,33.8,63.0,37.3,61.7
1,AL-02,Martha Roby (R),R+16,32.8,64.5,36.4,62.8
2,AL-03,Mike Rogers (R),R+16,31.9,64.8,36.7,62.2
3,AL-04,Robert Aderholt (R),R+30,17.3,79.8,23.9,74.7
4,AL-05,Mo Brooks (R),R+18,30.9,63.7,34.8,63.7


In [8]:
# Splits districts by state and district number. Also converts states to two letter abbreviations if not already.
def split_district(table, name):
    
    us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
    }

    state = []
    num = []
        
    if name == "pvi":
        table["split"] = table["Dist"].str.split("-", n=2)
        
        for index, row in table.iterrows():
            state.append(row['split'][0])
            if row['split'][1] == "AL":
                    num.append(0)
            else: num.append(int(row['split'][1]))

    if name == "ratings":
        table["split"] = table["District"].str.split("\xa0", n=2)
        
        for index, row in table.iterrows():
            state.append(us_state_abbrev[row['split'][0]])
            if row['split'][1] == "at-large":
                    num.append(0)
            else: num.append(int(row['split'][1]))
        
    table['state'] = state
    table['num'] = num

    return table

In [9]:
#Apply split_district to pvi data
pvi_fixed = split_district(pvi, "pvi")

pvi_fixed.head()

,Dist,Incumbent,PVI,Clinton %,Trump %,Obama %,Romney %,split,state,num
0,AL-01,Bradley Byrne (R),R+15,33.8,63.0,37.3,61.7,"[AL, 01]",AL,1
1,AL-02,Martha Roby (R),R+16,32.8,64.5,36.4,62.8,"[AL, 02]",AL,2
2,AL-03,Mike Rogers (R),R+16,31.9,64.8,36.7,62.2,"[AL, 03]",AL,3
3,AL-04,Robert Aderholt (R),R+30,17.3,79.8,23.9,74.7,"[AL, 04]",AL,4
4,AL-05,Mo Brooks (R),R+18,30.9,63.7,34.8,63.7,"[AL, 05]",AL,5


In [10]:
# Apply split_district to ratings data
ratings_fixed = split_district(ratings, "ratings")

ratings_fixed.head()

,District,2017 CPVI,Incumbent,Last result,Cook,IE,Sabato,Politico,Daily Kos,RCP,...,Safe D,Likely D,Lean D,Tilt D,Consensus,Max,Confidence,split,state,num
0,Alaska at-large,R+9,Don Young (R),53.1% R,Lean R,Likely R,Likely R,Likely R,Likely R,Likely R,...,0,0,0,0,Likely R,5,0.714286,"[Alaska, at-large]",AK,0
1,Arizona 1,R+2,Tom O'Halleran (D),53.8% D,Lean D,Likely D,Likely D,Lean D,Lean D,Lean D,...,1,2,4,0,Lean D,4,0.571429,"[Arizona, 1]",AZ,1
2,Arizona 2,R+1,Ann Kirkpatrick (D),54.7% D,Safe D,Safe D,Likely D,Likely D,Safe D,Lean D,...,4,2,1,0,Safe D,4,0.571429,"[Arizona, 2]",AZ,2
3,Arizona 6,R+9,David Schweikert (R),55.2% R,Tossup,Lean R,Lean R,Lean R,Lean R,Lean R,...,0,0,0,0,Lean R,5,0.714286,"[Arizona, 6]",AZ,6
4,Arizona 8,R+13,Debbie Lesko (R),55.5% R,Safe R,Safe R,Safe R,Likely R,Safe R,Safe R,...,0,0,0,0,Safe R,6,0.857143,"[Arizona, 8]",AZ,8


In [11]:
# Produce final data set used in visual
combined = pvi_fixed.merge(ratings_fixed, how='left', on=['state', 'num'])

combined.to_csv('combined.csv', index = False)

combined.head()

,Dist,Incumbent_x,PVI,Clinton %,Trump %,Obama %,Romney %,split_x,state,num,...,Tilt R,Tossup,Safe D,Likely D,Lean D,Tilt D,Consensus,Max,Confidence,split_y
0,AL-01,Bradley Byrne (R),R+15,33.8,63.0,37.3,61.7,"[AL, 01]",AL,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AL-02,Martha Roby (R),R+16,32.8,64.5,36.4,62.8,"[AL, 02]",AL,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AL-03,Mike Rogers (R),R+16,31.9,64.8,36.7,62.2,"[AL, 03]",AL,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AL-04,Robert Aderholt (R),R+30,17.3,79.8,23.9,74.7,"[AL, 04]",AL,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AL-05,Mo Brooks (R),R+18,30.9,63.7,34.8,63.7,"[AL, 05]",AL,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Dashboard created by: Michael Dunphy (he/him/his)

[Twitter](https://twitter.com/mtdunphy)
[Github](https://github.com/mtdunphy-umd)
[Tableau](https://public.tableau.com/profile/michael.dunphy8764)